In [503]:
import pandas as pd
from PandasUtils import *


In [504]:
data_dir = '/Users/j9/Desktop/metadata-mgmt-tool/pickled/datasets/'

df_pubtracker = pd.read_csv(data_dir + 'mta_pub_tracker.csv')
df_field_defs = pd.read_csv(data_dir +'mta_field_defs.csv')
df_master_dd = pd.read_csv(data_dir + '_alpha__Field_Dictionary_for_Open_Data_Portal_Datasets.csv')
df_field_list = pd.read_csv(data_dir + 'mta_field_list.csv')


In [505]:
df_field_defs = df_field_defs[['Feature Dataset', 'Feature Class', 'Attribute', 'Attribute Definition']].reset_index(drop=True)
df_pubtracker = df_pubtracker[['fourXFour', 'Layer Name', 'INV ID', 'Socrata Dataset Name']].reset_index(drop=True)
df_master_dd = df_master_dd[df_master_dd['Department'] == 'Municipal Transportation Agency'].reset_index(drop=True)
df_master_dd = df_master_dd[['columnID', 'datasetID', 'inventoryID', 'Dataset Name', 'Field Name', 'API Key','Open Data Portal URL', 'Field Type']].reset_index(drop=True)
df_field_list =  df_field_list[df_field_list['DestinationAttribute'] != 'Do Not Publish'].reset_index(drop=True)
df_field_list = df_field_list[['Feature Dataset', 'Feature Class', 'SourceAttribute', 'DestinationAttribute', 'Definition']].reset_index(drop=True)

In [506]:
def fieldToLower(df, field):
    df[field] = df[field].map(lambda x: str(x).lower().strip())
    return df

df_field_defs['Attribute Definition'] = df_field_defs['Attribute Definition'].map(lambda x: str(x).strip())
df_master_dd['Field Name Lower'] =   df_master_dd['Field Name'].map(lambda x: str(x).strip().lower())
df_field_list['DestinationAttribute Lower'] =   df_field_list['DestinationAttribute'].map(lambda x: str(x).strip().lower())
df_field_defs = fieldToLower(df_field_defs, 'Attribute')
df_field_defs = fieldToLower(df_field_defs, 'Feature Class')
df_field_list = fieldToLower(df_field_list, 'SourceAttribute')
df_field_list = fieldToLower(df_field_list, 'Feature Class')
df_pubtracker['Layer Name'] = df_pubtracker['Layer Name'].map(lambda x: 'mta.' +str(x))


In [507]:
df_all = pd.merge(df_field_list,  df_field_defs, how='left', left_on=['Feature Class', 'SourceAttribute'], right_on=['Feature Class', 'Attribute'])
df_all = PandasUtils.fillNaWithBlank(df_all)


In [508]:
df_features_from_attribute_list = PandasUtils.groupbyCountStar(df_field_list, 'Feature Class')
feature_list = list(df_pubtracker['Layer Name'])
print feature_list
df_all =  df_all[df_all['Feature Class'].isin(feature_list)].reset_index(drop=True)

['mta.bikecounters', 'mta.bikeparking', 'mta.bikeshare_stations', 'mta.bikevolume_manual', 'mta.bikewaynetwork', 'mta.greenwave_signals', 'mta.greenwave_streets', 'mta.pco_beats', 'mta.transitonlylane_violations', 'mta.carshare_onstreet', 'mta.colorcurb_blue_accessible', 'mta.meters', 'mta.meters_opsched', 'mta.meters_ratesched', 'mta.motorcycleparking_unmetered', 'mta.meteredblockfaces', 'mta.parkingcensus_offstreet', 'mta.parkingcensus_onstreet', 'mta.parkingregulations', 'mta.residentialpermitparking_areas', 'mta.residentialpermitparking_eligibility', 'mta.crossingguard_intersections', 'mta.crossingguard_schools', 'mta.paintedsafetyzones', 'mta.pedvolumemodel', 'mta.saferoutes_schools', 'mta.WalkFirst_intersections', 'mta.clearanceheights', 'mta.schools_15mph', 'mta.signals', 'mta.speedbumps', 'mta.speedlimits', 'mta.stopsigns', 'mta.trafficcircles', 'mta.vehiclerestrictions']


### List of Datasets with Field Count on the Open Data Portal

In [509]:
df_all_grp = PandasUtils.groupbyCountStar(df_all, 'Feature Class')
df_all_grp 

,Feature Class,count
0,mta.bikecounters,16
1,mta.bikeparking,10
2,mta.bikeshare_stations,19
3,mta.bikevolume_manual,17
4,mta.bikewaynetwork,30
5,mta.carshare_onstreet,33
6,mta.clearanceheights,6
7,mta.colorcurb_blue_accessible,10
8,mta.crossingguard_intersections,12
9,mta.crossingguard_schools,11


In [510]:
df_all_no_defs = df_all[ (df_all['Attribute'] == '') | (df_all['Definition'] == '') ]
df_all_defs =  df_all[(df_all['Attribute'] != '') | (df_all['Definition'] != '' )]

### Number of fields with defitions vs those without:

In [511]:
print "Fields with Definitions:"
print len(df_all_defs )
print "Count of Fields without definition"
print len(df_all_no_defs)

Fields with Definitions:
264
Count of Fields without definition
248


### Breakdown of Datasets  by number of fields with definitions

In [512]:
df_all_defs_grp = PandasUtils.groupbyCountStar(df_all_defs, 'Feature Class')
df_all_defs_grp

,Feature Class,count
0,mta.bikecounters,14
1,mta.bikeparking,8
2,mta.bikeshare_stations,10
3,mta.bikevolume_manual,15
4,mta.bikewaynetwork,21
5,mta.carshare_onstreet,28
6,mta.clearanceheights,2
7,mta.colorcurb_blue_accessible,1
8,mta.crossingguard_intersections,9
9,mta.crossingguard_schools,8


### Breakdown of Datasets  by number of fields with no definitions

In [513]:
df_all_no_defs_grp = PandasUtils.groupbyCountStar(df_all_no_defs, 'Feature Class')
df_all_no_defs_grp

,Feature Class,count
0,mta.bikecounters,2
1,mta.bikeparking,2
2,mta.bikeshare_stations,9
3,mta.bikevolume_manual,2
4,mta.bikewaynetwork,9
5,mta.carshare_onstreet,5
6,mta.clearanceheights,4
7,mta.colorcurb_blue_accessible,9
8,mta.crossingguard_intersections,3
9,mta.crossingguard_schools,3


### Look at specific datasets where Field Definitions are missing

In [514]:
#df_all[(df_all['Feature Class'] == 'mta.residentialpermitparking_eligibility') & (df_all['Definition'] == '')]

## Join the the fields to the master data dictionary to get the columnids

### Get columnids for fields that have definitions

In [515]:
df_all_defs = pd.merge(df_all_defs,  df_pubtracker, how='left', left_on='Feature Class', right_on='Layer Name')
#df_all_defs

In [516]:
df_all_defs_with_master_dd = pd.merge(df_all_defs,  df_master_dd, how='left', left_on=['DestinationAttribute Lower', 'fourXFour' ], right_on=['Field Name Lower', 'datasetID'])
df_all_defs_with_master_dd = PandasUtils.fillNaWithBlank(df_all_defs_with_master_dd)

In [517]:
df_all_defs_with_master_dd_no_columid = df_all_defs_with_master_dd[df_all_defs_with_master_dd['columnID'] == '']
df_all_defs_with_master_dd_columid = df_all_defs_with_master_dd[df_all_defs_with_master_dd['columnID'] != '']

In [518]:
print "Number of Fields that actually exist on the open data portal and have a field definition"
print len(df_all_defs_with_master_dd_columid)
print "Number of Fields that DONOT exist the open data portal but have a field definition"
print len(df_all_defs_with_master_dd_no_columid)

Number of Fields that actually exist on the open data portal and have a field definition
220
Number of Fields that DONOT exist the open data portal but have a field definition
44


### Get columnids for fields that DO NOT HAVE definitions

In [519]:
### Get the missing field defs
df_all_no_defs = pd.merge(df_all_no_defs,  df_pubtracker, how='left', left_on='Feature Class', right_on='Layer Name')
df_all_no_defs_with_master_dd = pd.merge(df_all_no_defs,  df_master_dd, how='left', left_on=['DestinationAttribute Lower', 'fourXFour' ], right_on=['Field Name Lower', 'datasetID'])
df_all_no_defs_with_master_dd = PandasUtils.fillNaWithBlank(df_all_no_defs_with_master_dd)

df_all_no_defs_with_master_dd_no_columid = df_all_no_defs_with_master_dd[df_all_no_defs_with_master_dd['columnID'] == '']
df_all_no_defs_with_master_dd_columid =df_all_no_defs_with_master_dd[df_all_no_defs_with_master_dd['columnID'] != '']


print "Number of Fields that actually exist on the open data portal with NO definition"
print len(df_all_no_defs_with_master_dd_columid)
print "Number of Fields that DONOT exist the open data portal and Don't have a definition"
print len(df_all_no_defs_with_master_dd_no_columid)

Number of Fields that actually exist on the open data portal with NO definition
175
Number of Fields that DONOT exist the open data portal and Don't have a definition
73


### Output the final csv to upload to Screendoor

In [520]:
#columnid	dataset_name	open_data_portal_url	field_name	field_type	field_definition	field_alias	 field_type_flag
#df_all_defs_with_master_dd_columid

df_final_defs = df_all_defs_with_master_dd_columid[['columnID', 'Dataset Name', 'Open Data Portal URL', 'Field Name', 'Field Type', 'Definition']].reset_index(drop=True)

df_final_defs['field_alias'] = ''
df_final_defs['field_type_flag'] = ''
#field_name_map = {'columnID': 'columnid', 'Dataset Name':'dataset_name', 'Open Data Portal URL': 'open_data_portal_url', 'Field Name': 'field_name', 'Field Type': 'field_type', 'Definition' : 'field_definition'}
field_name_map = {'Definition':'Field Definition'}
df_final_defs = PandasUtils.mapFieldNames(df_final_defs, field_name_map)
df_final_defs.to_csv(data_dir+"mta_field_defs_final", sep=',', encoding='utf-8')